# Имхасина, лаб3, генерация текста

In [ ]:
import tensorflow as tf

import numpy as np
import os
import time

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).



используемый текст - пушкин евгений онегин

In [ ]:
path_to_file = 'psh.txt'

### Read the data

In [ ]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print('Length of text: {} characters'.format(len(text)))

Length of text: 185030 characters


In [ ]:
# Take a look at the first 250 characters in text
print(text[:250])

		He мысля гордый свет забавить,
		Вниманье дружбы возлюбя,
		Хотел бы я тебе представить
		Залог достойнее тебя,
		Достойнее души прекрасной,
		Святой исполненной мечты,
		Поэзии живой и ясной,
		Высоких дум и простоты;
		Но так и быть – рукой прист


In [ ]:
# The unique characters in the file
vocab = sorted(set(text))
print('{} unique characters'.format(len(vocab)))

148 unique characters


## Process the text

необходимо отобразить строки в числовое представление. таблицы: одна отображает символы в числа, а другая - числа в символы

In [ ]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [ ]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  '\t':   0,
  '\n':   1,
  ' ' :   2,
  '!' :   3,
  '(' :   4,
  ')' :   5,
  '*' :   6,
  ',' :   7,
  '-' :   8,
  '.' :   9,
  '/' :  10,
  '0' :  11,
  '1' :  12,
  '2' :  13,
  '3' :  14,
  '4' :  15,
  '5' :  16,
  '6' :  17,
  '7' :  18,
  '8' :  19,
  ...
}


In [ ]:
# Show how the first 13 characters from the text are mapped to integers
print('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

'\t\tHe мысля го' ---- characters mapped to int ---- > [  0   0  33  57   2 120 135 125 119 139   2 111 122]


### The prediction task

tf.data.Dataset.from_tensor_slices - чтобы преобразовать текстовый вектор в поток индексов символов

In [ ]:
# The maximum length sentence you want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
    print(idx2char[i.numpy()])

	
	
H
e
 


In [ ]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
    print(repr(''.join(idx2char[item.numpy()])))

'\t\tHe мысля гордый свет забавить,\n\t\tВниманье дружбы возлюбя,\n\t\tХотел бы я тебе представить\n\t\tЗалог дос'
'тойнее тебя,\n\t\tДостойнее души прекрасной,\n\t\tСвятой исполненной мечты,\n\t\tПоэзии живой и ясной,\n\t\tВысок'
'их дум и простоты;\n\t\tНо так и быть – рукой пристрастной\n\t\tПрими собранье пестрых глав,\n\t\tПолусмешных,'
' полупечальных,\n\t\tПростонародных, идеальных,\n\t\tНебрежный плод моих забав,\n\t\tБессонниц, легких вдохнов'
'ений,\n\t\tНезрелых и увядших лет,\n\t\tУма холодных наблюдений\n\t\tИ сердца горестных замет.\n\n\n\nГлава первая'


In [ ]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [ ]:
for input_example, target_example in  dataset.take(1):
    print('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
    print('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  '\t\tHe мысля гордый свет забавить,\n\t\tВниманье дружбы возлюбя,\n\t\tХотел бы я тебе представить\n\t\tЗалог до'
Target data: '\tHe мысля гордый свет забавить,\n\t\tВниманье дружбы возлюбя,\n\t\tХотел бы я тебе представить\n\t\tЗалог дос'


In [ ]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 0 ('\t')
  expected output: 0 ('\t')
Step    1
  input: 0 ('\t')
  expected output: 33 ('H')
Step    2
  input: 33 ('H')
  expected output: 57 ('e')
Step    3
  input: 57 ('e')
  expected output: 2 (' ')
Step    4
  input: 2 (' ')
  expected output: 120 ('м')


### Create training batches


batch метод позволяет преобразовать  отдельные символы в последовательности желаемого размера.

In [ ]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

## Build The Model

In [ ]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

#GRU

tf.keras.layers.GRU : Тип RNN с размером units=rnn_units
добавим еще слой гру

In [ ]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                  batch_input_shape=[batch_size, None]),
        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model
    #recurrent_initializer='orthogonal'

In [ ]:
model = build_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

## Try the model


In [ ]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 148) # (batch_size, sequence_length, vocab_size)


In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           37888     
_________________________________________________________________
gru (GRU)                    (64, None, 1024)          3938304   
_________________________________________________________________
gru_1 (GRU)                  (64, None, 1024)          6297600   
_________________________________________________________________
dense (Dense)                (64, None, 148)           151700    
Total params: 10,425,492
Trainable params: 10,425,492
Non-trainable params: 0
_________________________________________________________________


Чтобы получить фактические прогнозы от модели,  необходимо выполнить выборку из выходного распределения, чтобы получить фактические индексы символов. Это распределение определяется логитами по словарю символов.

In [ ]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

Это дает нам предсказание каждого следующего символа

In [ ]:
sampled_indices

array([133,  88,  44,  95,  55,  68, 114,  45,  81,  64,  75, 146, 132,
         6,   3,  46,  95,  47, 118,  45,  46,  76,  81,  68,  90,  76,
        36,  24, 126,   4,  73,  46,  55,  70, 146,  65, 137,  17, 112,
        54,  65,  83, 115,  80,  33,  59,  79,   7,  31,  24, 124,  31,
        76,  70,  46,  83,  27, 121,  72,  52,  36,  46,  63,  73,  41,
        23, 141, 101,  45,  13,  44, 130,  63,  98,   3,  86,  67, 113,
        89,  91,  22,  83,  93, 134,  41,  67, 147, 134,  13,  28,  77,
         3, 126, 129, 110,  89, 138, 145, 102, 115])

При необученной модели:

In [ ]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 'входит в дом пустой,\n\t\tГде жил недавно наш герой.\n\t\tОна глядит: забытый в зале\n\t\tКий на бильярде отд'

Next Char Predictions: 
 'щЗTПcpжVАlw…ш*!WПXкVWxАpКxL>т(uWcr…mэ6дbmВз»Hg«,F>рFxrWВBнt_LWkuQ<–ХV2TцkТ!ЕoеИЛ;ВНъQo€ъ2Cy!тхвИю„Цз'


## Обучение

In [ ]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 148)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.9969015


In [ ]:
model.compile(optimizer='adam', loss=loss)

### Чек-поинты

In [ ]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

### Execute the training

In [ ]:
EPOCHS = 30

In [ ]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/30
28/28 [==============================] - 4s 156ms/step - loss: 3.9355
Epoch 2/30
28/28 [==============================] - 4s 154ms/step - loss: 2.9423
Epoch 3/30
28/28 [==============================] - 4s 155ms/step - loss: 2.5705
Epoch 4/30
28/28 [==============================] - 4s 154ms/step - loss: 2.4154
Epoch 5/30
28/28 [==============================] - 4s 154ms/step - loss: 2.3244
Epoch 6/30
28/28 [==============================] - 4s 155ms/step - loss: 2.2584
Epoch 7/30
28/28 [==============================] - 4s 155ms/step - loss: 2.1977
Epoch 8/30
28/28 [==============================] - 4s 154ms/step - loss: 2.1394
Epoch 9/30
28/28 [==============================] - 4s 156ms/step - loss: 2.0811
Epoch 10/30
28/28 [==============================] - 4s 155ms/step - loss: 2.0163
Epoch 11/30
28/28 [==============================] - 4s 155ms/step - loss: 1.9542
Epoch 12/30
28/28 [==============================] - 4s 157ms/step - loss: 1.8876
Epoch 13/30
28/28 [======

на 30ой итерации loss: 0.3193

## Генерация текста

### Restore the latest checkpoint

In [ ]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_30'

In [ ]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            37888     
_________________________________________________________________
gru_2 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
gru_3 (GRU)                  (1, None, 1024)           6297600   
_________________________________________________________________
dense_1 (Dense)              (1, None, 148)            151700    
Total params: 10,425,492
Trainable params: 10,425,492
Non-trainable params: 0
_________________________________________________________________


In [ ]:
def generate_text(model, start_string):
    # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    num_generate = 1000

    # Converting our start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperature results in more predictable text.
    # Higher temperature results in more surprising text.
    # Experiment to find the best setting.
    temperature = 1.0

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        # remove the batch dimension
        predictions = tf.squeeze(predictions, 0)

        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

        # Pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [ ]:
print(generate_text(model, start_string=u"А ты"))

А ты, милое!
		Высокой струею силится?
		Обминаденные моды,
		Но недоим к своим цветая, каковы, например: Агафон, Филат, Федора; Фекла и проч., употребляются. Голпава – жань пред собою
		Ожи
		Да случай обычною руки
		Наснушал б она возолжал;
		Всегда как луна была согрета.
		Прощаясь перед натих одну и топор!



XI. XIV

		Но здесь с подушки? ты ходет,
		Стеснялась каждочтою сладкой
		Не спится ей поне?
		Что все подружки измарали
		С Оленька семейства оживляй,
		В мой скользвинелай ла
		Как рано мог озаряя время расчислено пи вод,
		Но ясно бродит она внаманьем
		За глушилось.
		В округом сего в очам.
		Манежь приятель, тишина,
		В глуши лесов обедает двор
		И отверил е. Ленский сам
		Не верит собственным глазам.



XXI

		Снег пьердцым издалии злою
		В Молюбому,
		И верно б нос говорливой,
		И ей открылся мир иной.



XX

		Roosil пароем Тудски,
		Развеселить веном уком,
	Его Капридал,
		Читая друзья! С тайне, стного сенья
		Обозык отчатность маг.


* Сидит лиром следыми и пробка в 